# Multi-Agent Systems & Workflow Patterns

**Day 1B — Google ADK | Local VS Code**

You’ll build:
- Multi-agent teams
- Sequential, Parallel, Loop workflows
- LLM as manager

---
**No Kaggle. No extra installs. Just code.**

### Load API Key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Add GOOGLE_API_KEY to .env in project root!")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

print("Gemini API key loaded!")

Gemini API key loaded!


### Import ADK

In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search

print("ADK ready!")

ADK ready!


### Why Multi-Agent Systems?

Single agent = Do-it-all (unreliable for complex tasks)

Multi-agent = Team of specialists (easier to build/debug/powerful)

### Research Agent

In [3]:
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

print("✅ research_agent created.")

✅ research_agent created.


### Summarizer Agent

In [4]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


### Root Coordinator

In [5]:
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.


### Create Runner & Run

In [6]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("When is the next prelims of UPSC 2026?")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > When is the next prelims of UPSC 2026?


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


ResearchCoordinator > The UPSC Civil Services Preliminary Examination for 2026 is scheduled for May 24, 2026. The official notification is expected on January 14, 2026, with the application window open until February 3, 2026. The Mains examination will begin on August 21, 2026.


## Sequential Workflows - The Assembly Line

Use when: Order matters (e.g., Outline → Write → Edit)

#### Outline Agent

In [7]:
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's topic, create a detailed outline for a blog post.
    Include sections like Introduction, Main Points, and Conclusion.
    Output only the outline as a bulleted list.""",
    output_key="blog_outline",
)

print("✅ outline_agent created.")

✅ outline_agent created.


#### Writer Agent

In [8]:
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Using this outline: {blog_outline}
Write a full blog post draft. Make it engaging and around 400-600 words.
Output only the blog post text.""",
    output_key="blog_draft",
)

print("✅ writer_agent created.")

✅ writer_agent created.


#### Editor Agent

In [9]:
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Review this blog draft: {blog_draft}
Edit for clarity, grammar, flow, and engagement.
Output the final edited version.""",
    output_key="final_blog",
)

print("✅ editor_agent created.")

✅ editor_agent created.


### Sequential Root

In [10]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential root_agent created.")

✅ Sequential root_agent created.


#### Run Sequential

In [11]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the benefits of meditation for programmers")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a blog post about the benefits of meditation for programmers
OutlineAgent > *   **Introduction**
    *   Hook: The demanding nature of programming – long hours, complex problem-solving, constant learning.
    *   Introduce meditation as a potential tool for programmers.
    *   Briefly state the blog post's purpose: to explore the tangible benefits of meditation for those in the coding world.
    *   Thesis statement: Meditation can significantly enhance a programmer's cognitive abilities, emotional resilience, and overall well-being, leading to improved performance and a more sustainable career.

*   **Main Points**

    *   **Enhanced Focus and Concentration**
        *   The "flow state" in programming: deep concentration, immersion.
        *   How meditation trains the brain to resist distractions (internal and external).
        *   Specific meditation techniques beneficial for focus (e.g., breath awareness).
        *   R

## Parallel Workflows - The Research Team

Use when: Independent tasks (e.g., Multi-topic research)

In [12]:
#### Parallel Agents

tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="Research the latest in Tech. Use google_search. Output 3 key updates.",
    tools=[google_search],
    output_key="tech_report",
)

health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="Research the latest in Health. Use google_search. Output 3 key updates.",
    tools=[google_search],
    output_key="health_report",
)

finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="Research the latest in Finance. Use google_search. Output 3 key updates.",
    tools=[google_search],
    output_key="finance_report",
)

aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Compile these reports into a single executive briefing:
Tech: {tech_report}
Health: {health_report}
Finance: {finance_report}
Format as sections with headings.""",
    output_key="exec_briefing",
)

print("✅ Parallel agents created.")

✅ Parallel agents created.


### Parallel Team

In [13]:
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel + Sequential created.")

✅ Parallel + Sequential created.


### Run Parallel

In [14]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > Here are three key updates in Finance for November 2025:

*   **Federal Reserve's Cautious Easing Amid Economic Uncertainty:** The Federal Reserve has continued its rate cuts, influenced by weakening employment trends, despite ongoing inflation concerns. However, internal disagreements within the Fed are growing, with some officials advocating for faster cuts and others urging caution due to persistent inflation. The likelihood of a December rate cut is now uncertain, as Fed Chair Powell described the current policy-making environment as akin to "driving in fog," emphasizing a slower approach until clearer economic data emerges.

*   **Stablecoin Regulation on the Horizon:** The Bank of England has launched a consultation on a proposed regulatory regime for sterling-denominated systemic stablecoins. This initiative aims to prepare for a future where digi

## Loop Workflows - The Refinement Cycle

Use when: Iterative improvement (e.g., Writer + Critic loop)

### Loop Agents & Run

In [15]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",
)

critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",
)

def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story",
    tools=[FunctionTool(exit_loop)],
)

story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,
)

root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop agents created.")

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Loop agents created.

 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias, his beard perpetually flecked with salt spray, polished the great Fresnel lens. The kerosene lamp hummed its familiar tune, a melody against the gnawing Atlantic wind. Tonight, however, the rhythm was broken by a dull thud against the lighthouse base. He descended the spiral stairs, oil lamp casting dancing shadows.

On the rocky shore, nestled amongst seaweed and barnacles, lay a rolled parchment, strangely untouched by the sea. Unfurling it, Elias gasped. It wasn't paper, but a material that shimmered, shifting with an internal, ethereal light. Etched upon its surface were lines and symbols he’d never seen, a celestial chart perhaps, or a map of some forgotten realm. As he traced a pulsing, blue current, a faint warmth spread through his fingertips. The map glowed brighter, its luminescence a silen